In [1]:
import pandas as pd , os, numpy as np, matplotlib.pyplot as plt , glob, re
from sklearn.model_selection import train_test_split

from nltk.tokenize import regexp_tokenize ,word_tokenize ,sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.moses import MosesDetokenizer
from nltk.corpus import stopwords
from gensim.corpora.dictionary import Dictionary

from keras.preprocessing import sequence
from keras.layers import Embedding, Dropout,Conv1D,GlobalMaxPooling1D , Activation,Dense
from keras.models import Sequential
import keras

%precision 2
%matplotlib inline

Using TensorFlow backend.


In [2]:
fpath = glob.glob(os.path.join(os.getcwd(),'smsspam/*'))[0]      
with open(fpath) as f:
    print '***************************************************************'
    print f.readline()
    print '***************************************************************'
    lines = [line.strip().split('\t') for  line in f.readlines()]
y, X = [ int(line[0]=='spam')for line in lines] , [line[1] for line in lines]       


***************************************************************
ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...

***************************************************************


In [3]:
text_len = np.array(map(len,X)); 
text_len.max(), text_len.min(), text_len.mean() , np.median(text_len)

(910, 2, 80.38, 61.00)

In [4]:
stop = set(stopwords.words('english'))
detokenizer = MosesDetokenizer()
lem = WordNetLemmatizer()

In [5]:
X1= [detokenizer.detokenize([lem.lemmatize(i) for i in regexp_tokenize(X[list], r'\w+\.?|\d+') if i not in stop] ,return_str=True)  for list in range(len(X)) ] 

In [6]:
dictionary = Dictionary([word_tokenize(X1[i])for i in range(len(X1))])

In [7]:
word2idx = {value :idx for idx, value in dictionary.items()}

In [8]:
X2 = [[word2idx[i] for i in word_tokenize(X1[list])] for list in range(len(X1))]

In [9]:
X_train, X_test,y_train, y_test = train_test_split(X2, y,random_state=57,shuffle=True,stratify=y,test_size=0.3 )

In [10]:
x_train = sequence.pad_sequences(X_train, maxlen=150)
x_test = sequence.pad_sequences(X_test, maxlen=150)

In [11]:
model = Sequential([
    Embedding(10579,50, input_length=150 ),
    Dropout(0.2),
    Conv1D(250, 3 , padding='valid', activation='relu', strides=1),
    GlobalMaxPooling1D(),
    Dense(250),
    Dropout(0.2),
    Activation('relu'),
    Dense(1),
    Activation('sigmoid')
   
])

In [12]:
#SGD = keras.optimizers.SGD(lr=0.001)

In [13]:
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [14]:
model.fit(x_train, y_train, 
         batch_size=57,
         epochs=8,
         validation_data=(x_test, y_test))

Train on 3901 samples, validate on 1672 samples
Epoch 1/8
3901/3901 [==============================] - 15s - loss: 0.4090 - acc: 0.8647 - val_loss: 0.3278 - val_acc: 0.8660